# Multi-Agent System with Per-Agent System Messages

## Why a System Message per agent?

In a multi-agent system, **each agent has a specific role**. Giving a distinct System Message to each agent allows:

1. **Clear specialization**: Each agent knows precisely its role and responsibilities
2. **Separation of concerns**: Prevents agents from interfering with each other's work
3. **Optimal quality**: Each agent is "configured" to excel at its specific task
4. **Differentiated behaviors**: Allows different styles and detail levels (concise planner, detailed solver, strict critic)

## When to use separate System Messages?

✅ **Use when:**
- You have **several distinct roles** (planning, executing, validating, synthesizing)
- Each agent requires **different behavior** (creative vs analytical, concise vs detailed)
- You want to **avoid role confusion** (a solver that plans, a planner that solves)
- The workflow is **complex with several** specialized steps
- You need to **strictly constrain** each agent (e.g., critic responds "OK" only)

❌ **Avoid when:**
- All agents do **the same thing** (no specialization)
- The workflow is **simple and linear** without need for differentiated roles
- You want **flexibility** between roles

## Best practices:

✨ **DO:**
- Be **explicit and concise** in each System Message
- Clearly define **what the agent MUST do** and **what it MUST NOT do**
- Use clear **action verbs** (break, follow, evaluate, synthesize)
- Specify the **expected output format** if necessary (e.g., "Reply ONLY with OK or...")
- Name agents **descriptively** (Planner, Solver, Critic rather than Agent1, Agent2)

⚠️ **DON'T:**
- Messages too long or ambiguous
- Duplicate the same message for all agents
- Give multiple conflicting responsibilities to the same agent
- Forget to specify important constraints (format, length, style)

In [ ]:
from typing import TypedDict, List
from langchain_core.messages import (
    BaseMessage,
    HumanMessage,
    SystemMessage,
    AIMessage,
)
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, END

from dotenv import load_dotenv

load_dotenv()


In [ ]:
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0
)


In [ ]:
class AgentState(TypedDict):
    messages: List[BaseMessage]
    retries: int


In [ ]:
PLANNER_SYSTEM = SystemMessage(
    content=(
        "You are a planning agent.\n"
        "Your job is to break the user request into clear steps.\n"
        "Do NOT answer the question."
    )
)

SOLVER_SYSTEM = SystemMessage(
    content=(
        "You are a solver agent.\n"
        "Follow the plan strictly and produce a clear answer."
    )
)

CRITIC_SYSTEM = SystemMessage(
    content=(
        "You are a critic agent.\n"
        "Evaluate the answer.\n"
        "Reply ONLY with OK or a short critique."
    )
)


## Agent 1 — Planner 🗺️

In [ ]:
def planner_node(state: AgentState) -> AgentState:
    response = llm.invoke([
        PLANNER_SYSTEM,
        *state["messages"]
    ])

    return {
        "messages": state["messages"] + [
            AIMessage(content=f"PLAN:\n{response.content}")
        ],
        "retries": state["retries"]
    }


## Agent 2 — Solver ✍️

In [ ]:
def solver_node(state: AgentState) -> AgentState:
    response = llm.invoke([
        SOLVER_SYSTEM,
        *state["messages"]
    ])

    return {
        "messages": state["messages"] + [
            AIMessage(content=f"ANSWER:\n{response.content}")
        ],
        "retries": state["retries"]
    }


## Agent 3 — Critic ✅

In [ ]:
def critic_node(state: AgentState) -> AgentState:
    last_answer = state["messages"][-1].content

    critique = llm.invoke([
        CRITIC_SYSTEM,
        HumanMessage(content=last_answer)
    ])

    return {
        "messages": state["messages"] + [critique],
        "retries": state["retries"] + (0 if critique.content.strip() == "OK" else 1)
    }


## Decision: retry or end 🔁

In [ ]:
def should_retry(state: AgentState) -> str:
    if state["retries"] >= 2:
        return "end"

    return "retry" if state["messages"][-1].content != "OK" else "end"

## Build the multi-agent graph

In [ ]:
graph = StateGraph(AgentState)

graph.add_node("planner", planner_node)
graph.add_node("solver", solver_node)
graph.add_node("critic", critic_node)

graph.set_entry_point("planner")

graph.add_edge("planner", "solver")
graph.add_edge("solver", "critic")

graph.add_conditional_edges(
    "critic",
    should_retry,
    {
        "retry": "solver",
        "end": END
    }
)

app = graph.compile()


In [ ]:
from IPython.display import display, Image

display(Image(app.get_graph().draw_mermaid_png()))

In [ ]:
result = app.invoke({
    "messages": [
        HumanMessage(content="Explain LangGraph simply")
    ],
    "retries": 0
})

for m in result["messages"]:
    print(m.type.upper(), "→", m.content)
